<b> Assignment 1 </b>  
Raphael Antioquia  
031379126  
CVI520NSB  
Ellie Azizi  

<b>Part I: A photo booth application</b>  
  
Recreate the OpenCV logo using OpenCV drawing functions only (e.g., cv2.circle, cv2.line, etc.).  
  
You must draw:  
  
• Three colored shapes (ellipse: Blue, Green, Red) arranged in a triangular pattern.  
• Proper positioning and size of circles.  
• The text "OpenCV" at the center/bottom.  

In [2]:
import cv2
import numpy as np

# Create a white canvas of size 500x500 with 3 color channels (RGB)
image = np.full((500, 500, 3), 255, dtype=np.uint8)

# Center point of the canvas
center_x, center_y = 250, 250

# Radii for outer and inner ellipses
outer_radius = 90
inner_radius = 30

# Red ellipse (top center)
cv2.ellipse(image, (center_x, center_y - 110), (outer_radius, outer_radius), 120, 0, 300, (0, 0, 255), -1)
cv2.ellipse(image, (center_x, center_y - 110), (inner_radius, inner_radius), 120, 0, 300, (255, 255, 255), -1)

# Green ellipse (bottom left)
cv2.ellipse(image, (center_x - 98, center_y + 48), (outer_radius, outer_radius), 0, 0, 300, (0, 255, 0), -1)
cv2.ellipse(image, (center_x - 98, center_y + 48), (inner_radius, inner_radius), 0, 0, 300, (255, 255, 255), -1)

# Blue ellipse (bottom right)
cv2.ellipse(image, (center_x + 98, center_y + 48), (outer_radius, outer_radius), 300, 0, 300, (255, 0, 0), -1)
cv2.ellipse(image, (center_x + 98, center_y + 48), (inner_radius, inner_radius), 300, 0, 300, (255, 255, 255), -1)

# Add the text "OpenCV" near the top (adjusted for centering)
cv2.putText(image, 'OpenCV', (center_x - 98, center_y + 200), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 0), 6)

# Save and display the image
cv2.imwrite("data/opencv-logo.png", image)
cv2.imshow("Image", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

<b>Part II: Image Arithmetic </b>  
Write a Python script to manually blend two images using NumPy operations.  
  
Use the following formula to blend them:  
  
blend = (1−α)⋅img1 + α⋅img2  
  
• alpha must be a value between 0 and 1.  
• Do not use cv2.addWeighted.  
• Display and save the blended image as "manual_blend.jpg"

In [3]:
import cv2
import numpy as np

# Load the images
img1 = cv2.imread('cat1.jpg')
img2 = cv2.imread('cat2.jpg')

# Resize img2 to match img1 dimensions if needed
if img1.shape != img2.shape:
    img2 = cv2.resize(img2, (img1.shape[1], img1.shape[0]))

# Define alpha 
alpha = 0.5

# Blend the images manually
blended = (1 - alpha) * img1 + alpha * img2
blended = np.clip(blended, 0, 255).astype(np.uint8)

# Save the blended image
cv2.imwrite('manual_blend.jpg', blended)

cv2.imshow("Blended Image", blended)
cv2.waitKey(0)
cv2.destroyAllWindows()

<b>Part III: Image Arithmetic</b>  
  
Design a modular photo editing application in Python using OpenCV and NumPy. The app should allow  
users to load an image and apply a sequence of processing steps interactively through a menu interface  

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Initialize history stacks to enable undo and log operations
image_history = []
action_history = []

def record_action(img, action_text):
    # Save a copy of the current image and record the action description
    image_history.append(img.copy())
    action_history.append(action_text)

def bgr_to_rgb(img):
    # Convert BGR format (default in OpenCV) to RGB format for correct display with matplotlib
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

def show_side_by_side(original, edited, title):
    # Display the original and the edited image side by side for visual comparison
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(bgr_to_rgb(original))
    plt.title("Original")
    plt.axis("off")

    plt.subplot(1, 2, 2)
    if len(edited.shape) == 2:  # grayscale
        plt.imshow(edited, cmap='gray')
    else:  # color
        plt.imshow(bgr_to_rgb(edited))
    plt.title(title)
    plt.axis("off")

    plt.tight_layout()
    plt.show()

def adjust_brightness(img):
    # Adjust the image brightness using a user-specified beta value
    while True:
        try:
            beta = int(input("\nEnter brightness value (e.g. -100 to 100): "))
            break
        except ValueError:
            print("\nPlease enter a valid integer.")
    result = cv2.convertScaleAbs(img, alpha=1.0, beta=beta)
    show_side_by_side(img, result, f"Brightness {beta}")
    return result, f"brightness {beta}"

def adjust_contrast(img):
    # Adjust the image contrast using a user-specified alpha multiplier
    while True:
        try:
            alpha = float(input("\nEnter contrast scale (e.g. 0.5 to 3.0): "))
            break
        except ValueError:
            print("\nPlease enter a valid number.")
    result = cv2.convertScaleAbs(img, alpha=alpha, beta=0)
    show_side_by_side(img, result, f"Contrast x{alpha}")
    return result, f"contrast x{alpha}"

def convert_to_grayscale(img):
    # Convert a color image to grayscale
    result = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    show_side_by_side(img, result, "Grayscale")
    return result, "converted to grayscale"

def add_padding(img):
    # Add padding to the image based on user-specified size and target aspect ratio
    h, w = img.shape[:2]
    while True:
        try:
            padding_size = int(input("Enter base padding size: "))
            if padding_size >= 0:
                break
            else:
                print("Must be non-negative.")
        except ValueError:
            print("Enter a valid integer.")

    # Ask user to choose border style
    border_types = {'1': cv2.BORDER_CONSTANT, '2': cv2.BORDER_REFLECT, '3': cv2.BORDER_REPLICATE}
    print("\nBorder type:\n1. constant\n2. reflect\n3. replicate (Enter for default)")
    border_choice = input("Choice: ").strip()
    border_type = border_types.get(border_choice, cv2.BORDER_CONSTANT)

    # Ask user for desired aspect ratio
    print("\nProportion:\n1. Square (1:1)\n2. Rectangle (16:9)\n3. Custom")
    while True:
        rc = input("\nChoice (1-3): ").strip()
        if rc == '1':
            target_ratio = 1.0
            break
        elif rc == '2':
            target_ratio = 16 / 9
            break
        elif rc == '3':
            try:
                r1, r2 = map(int, input("\nEnter custom ratio (e.g. 4:5): ").split(":"))
                target_ratio = r1 / r2
                break
            except:
                print("\nInvalid format.")
        else:
            print("\nEnter 1, 2, or 3.")

    # Calculate padding amounts to match the target aspect ratio
    current_ratio = w / h
    top = bottom = left = right = 0
    if current_ratio < target_ratio:
        new_width = int(h * target_ratio)
        pad = new_width - w
        left = pad // 2
        right = pad - left
    else:
        new_height = int(w / target_ratio)
        pad = new_height - h
        top = pad // 2
        bottom = pad - top

    # Add the user-specified padding size
    top += padding_size
    bottom += padding_size
    left += padding_size
    right += padding_size

    # Apply padding using OpenCV
    result = cv2.copyMakeBorder(img, top, bottom, left, right, border_type, value=[0, 0, 0])
    show_side_by_side(img, result, "Padded")
    return result, f"padded {padding_size}px with type {border_choice or '1'}"

def apply_thresholding(img):
      # Ensure image is 3-channel before converting to grayscale
    if len(img.shape) == 2:
        gray = img  # Already grayscale
    else:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
    print("\nThresholding:\n1. Binary\n2. Inverted")
    while True:
        t_type = input("\nChoose (1 or 2): ")
        if t_type == '1':
            t_mode = cv2.THRESH_BINARY
            desc = "binary"
            break
        elif t_type == '2':
            t_mode = cv2.THRESH_BINARY_INV
            desc = "binary inverse"
            break
        else:
            print("\nInvalid input.")
    while True:
        try:
            thresh = int(input("\nThreshold value (0-255): "))
            if 0 <= thresh <= 255:
                break
        except ValueError:
            print("\nEnter a valid number.")

    # Apply OpenCV thresholding function
    _, result = cv2.threshold(gray, thresh, 255, t_mode)
    show_side_by_side(img, result, f"Threshold {desc} {thresh}")
    return result, f"threshold {desc} {thresh}"

def blend_images(img1):
    # Ask user for second image and blend it with the current one using a specified alpha
    while True:
        path = input("\nSecond image path: ").strip()
        img2 = cv2.imread(path)
        if img2 is not None:
            break
        print("\nCouldn't load image.")

     # Convert grayscale to BGR if needed
    if len(img1.shape) == 2 and len(img2.shape) == 3:
        img1 = cv2.cvtColor(img1, cv2.COLOR_GRAY2BGR)
    elif len(img1.shape) == 3 and len(img2.shape) == 2:
        img2 = cv2.cvtColor(img2, cv2.COLOR_GRAY2BGR)

    # Resize second image to match dimensions of the first if needed
    if img1.shape != img2.shape:
        img2 = cv2.resize(img2, (img1.shape[1], img1.shape[0]))

    while True:
        try:
            alpha = float(input("\nAlpha (0-1): "))
            if 0 <= alpha <= 1:
                break
        except ValueError:
            print("\nEnter a valid float.")

    # Perform manual image blending using numpy
    blended = (1 - alpha) * img1 + alpha * img2
    blended = np.clip(blended, 0, 255).astype(np.uint8)
    show_side_by_side(img1, blended, f"Blended alpha {alpha}")
    return blended, f"blended with alpha {alpha}"

def undo_last_operation():
    # Undo the last operation by reverting to the previous image state
    if image_history:
        return image_history.pop()
    else:
        print("\nNothing to undo.")
        return None

def show_action_history():
    # Print a list of all actions taken during the session
    if action_history:
        print("\n== History ==")
        for i, act in enumerate(action_history, 1):
            print(f"{i}. {act}")
    else:
        print("\nNo history.")

def main():
    img = cv2.imread('cat1.jpg') # Default cat image to load if present

    # If defautl cat image is not present, loop until a valid image is loaded
    while img is None:
        print("Default cat image is not present in directory.")
        path = input("Enter path to image: ").strip()
        img = cv2.imread(path)
        if img is None:
            print("Could not load image. Please try again.")

    while True:
        print("""
\n==== Mini Photo Editor ====
1. Adjust Brightness
2. Adjust Contrast
3. Convert to Grayscale
4. Add Padding (choose border type)
5. Apply Thresholding (binary or inverse)
6. Blend with Another Image (manual alpha)
7. Undo Last Operation
8. View History of Operations
9. Save and Exit
""")
        choice = input("Select an option (1-9): ").strip()

        if choice == '1':
            record_action(img, "brightness")
            img, desc = adjust_brightness(img)
            action_history[-1] = desc

        elif choice == '2':
            record_action(img, "contrast")
            img, desc = adjust_contrast(img)
            action_history[-1] = desc

        elif choice == '3':
            record_action(img, "grayscale")
            img, desc = convert_to_grayscale(img)
            action_history[-1] = desc

        elif choice == '4':
            record_action(img, "padding")
            img, desc = add_padding(img)
            action_history[-1] = desc

        elif choice == '5':
            record_action(img, "threshold")
            img, desc = apply_thresholding(img)
            action_history[-1] = desc

        elif choice == '6':
            record_action(img, "blend")
            img, desc = blend_images(img)
            action_history[-1] = desc

        elif choice == '7':
            restored = undo_last_operation()
            if restored is not None:
                img = restored
                action_history.pop()

        elif choice == '8':
            show_action_history()

        elif choice == '9':
            while True:
                fname = input("\nSave as filename (e.g., output.jpg or output.png): ")
                if fname.lower().endswith(('.jpg', '.jpeg', '.png')):
                    cv2.imwrite(fname, img)
                    print(f"\nSaved as {fname}.")
                    show_action_history()
                    break
                else:
                    print("\nFilename must end with .jpg, .jpeg, or .png")
            break

        else:
            print("\nInvalid option. Try again.")

if __name__ == '__main__':
    main()